# **Pracownicy**

Zadanie dot. rozdysponowania godzin pracowników w sklepie.

![Pracownicy](https://raw.githubusercontent.com/ptrxpl/missp-final/main/misc/pracownicy.png)

Należy uzupełnić kalendarz zmian tak, aby zminimalizować koszta firmy.


# **Analiza zadania**
1. Zmienne decyzyjne
2. Funkcja celu
3. Ograniczenia
4. Rozwiązanie

**Zmienne**

Kto i kiedy jest dostępny?


```
tablica = LpVariable.dicts("",((i, j) for i in dni_tygodnia for j in osoby),cat='Binary')
```

**Funkcja celu**

Minimalizacja wypłaty 


```
prob += lpSum(tablica[(i,j)] * stawki[j] for i in dni_tygodnia for j in osoby)
```

**Ograniczenia**

Pracownik jest dostępny minimum raz a maksymalnie 3 razy w tygodniu:


```
for osoba in osoby:
  prob += lpSum(tablica[(i,osoba)] for i in dni_tygodnia) <= 3
  prob += lpSum(tablica[(i,osoba)] for i in dni_tygodnia) >= 1 
```

Pracownik pracuje tylko wtedy, kiedy jest dostępny:

```
for osoba in osoby:
  prob += lpSum(tablica[(i,osoba)] for i in dni_tygodnia if i not in dostepne_dni_danej_osoby[osoba]) == 0
```

Zapotrzebowanie osób w konkretne dni:


```
for dzien in dni_tygodnia:
  prob += lpSum(tablica[(dzien,i)] for i in osoby) == liczba_osob_danego_dnia[dzien]
```

**Rozwiązanie**

Stworzono 2 wersje: bez oraz z wczytywania pliku csv (Excel).











In [2]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

# Link pomocniczy:
# https://benalexkeen.com/linear-programming-with-python-and-pulp-part-4/

Reading package lists... Done
Building dependency tree       
Reading state information... Done
coinor-cbc is already the newest version (2.9.9+repack1-1).
coinor-clp is already the newest version (1.16.11+repack1-1).
glpk-utils is already the newest version (4.65-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


**Poniżej wersja bez Excela:**

In [3]:
# Wersja BEZ excela

from pulp import *

prob = LpProblem("Pracownicy_bez_csv", LpMinimize)

# lista dni tygodnia
dni_tygodnia = ['Poniedziałek', 'Wtorek', 'Środa', 'Czwartek', 'Piątek']
osoby = ['Ania', 'Stefan', 'Hektor', 'Olaf', 'Lidia', 'Piotr']
liczba_osob_danego_dnia = {'Poniedziałek': 2, 
               'Wtorek': 1, 
               'Środa': 1, 
               'Czwartek': 1, 
               'Piątek': 3}
dostepne_dni_danej_osoby = {'Ania': ['Poniedziałek', 'Wtorek', 'Środa', 'Czwartek', 'Piątek'],
               'Stefan': ['Poniedziałek'],
               'Hektor': ['Poniedziałek', 'Wtorek', 'Środa'],
               'Olaf': ['Poniedziałek', 'Wtorek', 'Środa', 'Czwartek', 'Piątek'],
               'Lidia': ['Czwartek', 'Piątek'],
               'Piotr': ['Poniedziałek', 'Wtorek', 'Środa']}
stawki = {'Ania':100,'Stefan':50,'Hektor':60,'Olaf':40,'Lidia':11,'Piotr':70}

# A dictionary called 'dni_tygodnia' is created to contain the referenced Variables
tablica = LpVariable.dicts("",((i, j) for i in dni_tygodnia for j in osoby),cat='Binary')

prob += lpSum(tablica[(i,j)] * stawki[j] for i in dni_tygodnia for j in osoby)
for dzien in dni_tygodnia:
  prob += lpSum(tablica[(dzien,i)] for i in osoby) == liczba_osob_danego_dnia[dzien]
for osoba in osoby:
  prob += lpSum(tablica[(i,osoba)] for i in dni_tygodnia) <= 3
  prob += lpSum(tablica[(i,osoba)] for i in dni_tygodnia) >= 1 
for osoba in osoby:
  prob += lpSum(tablica[(i,osoba)] for i in dni_tygodnia if i not in dostepne_dni_danej_osoby[osoba]) == 0

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The optimised objective function value is printed to the screen    
print ("Firmę będzie to kosztować w tygodniu = ", value(prob.objective),"zł")
print("Plan na tydzień dla poszczególnych osób")
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print (v.name, "=", v.varValue)


Firmę będzie to kosztować w tygodniu =  382.0 zł
Plan na tydzień dla poszczególnych osób
_('Czwartek',_'Ania') = 0.0
_('Czwartek',_'Hektor') = 0.0
_('Czwartek',_'Lidia') = 1.0
_('Czwartek',_'Olaf') = 0.0
_('Czwartek',_'Piotr') = 0.0
_('Czwartek',_'Stefan') = 0.0
_('Piątek',_'Ania') = 1.0
_('Piątek',_'Hektor') = 0.0
_('Piątek',_'Lidia') = 1.0
_('Piątek',_'Olaf') = 1.0
_('Piątek',_'Piotr') = 0.0
_('Piątek',_'Stefan') = 0.0
_('Poniedziałek',_'Ania') = 0.0
_('Poniedziałek',_'Hektor') = 0.0
_('Poniedziałek',_'Lidia') = 0.0
_('Poniedziałek',_'Olaf') = 1.0
_('Poniedziałek',_'Piotr') = 0.0
_('Poniedziałek',_'Stefan') = 1.0
_('Wtorek',_'Ania') = 0.0
_('Wtorek',_'Hektor') = 1.0
_('Wtorek',_'Lidia') = 0.0
_('Wtorek',_'Olaf') = 0.0
_('Wtorek',_'Piotr') = 0.0
_('Wtorek',_'Stefan') = 0.0
_('Środa',_'Ania') = 0.0
_('Środa',_'Hektor') = 0.0
_('Środa',_'Lidia') = 0.0
_('Środa',_'Olaf') = 0.0
_('Środa',_'Piotr') = 1.0
_('Środa',_'Stefan') = 0.0


**Poniżej wersja z Excelem:**

In [7]:
# Wersja z excelem

import pandas as pd
from pulp import *

dane = pd.read_csv('https://raw.githubusercontent.com/ptrxpl/missp-final/main/misc/pracownicy.csv', sep=';',index_col = 'Imiona')
dane_dni_tygodnia = pd.DataFrame(dane, columns = ['Poniedzialek', 'Wtorek', 'Sroda', 'Czwartek', 'Piatek'])
dane_kwota = pd.DataFrame(dane, columns = ['kwota/h'])
dane_liczba_osob_danego_dnia = dane.loc[['liczba osob danego dnia'],['Poniedzialek', 'Wtorek', 'Sroda', 'Czwartek', 'Piatek']]
dane_dni_tygodnia = dane_dni_tygodnia.drop('liczba osob danego dnia')

prob = LpProblem("Pracownicy_z_csv", LpMinimize)
tablica = LpVariable.dicts("",((dzien, osoba) for dzien in dane_dni_tygodnia.columns for osoba in dane_dni_tygodnia.index),cat='Binary')
#print(int(dane_liczba_osob_danego_dnia.loc['liczba osob danego dnia','Poniedzialek']))

prob += lpSum(tablica[(i,j)] * int(dane_kwota.loc[j,'kwota/h']) for i in dane_dni_tygodnia.columns for j in dane_dni_tygodnia.index)
for dzien in dane_dni_tygodnia.columns:
    prob += lpSum(tablica[(dzien,osoba)] for osoba in dane_dni_tygodnia.index) == int(dane_liczba_osob_danego_dnia.loc['liczba osob danego dnia',dzien])
for osoba in dane_dni_tygodnia.index:
    prob += lpSum(tablica[(i,osoba)] for i in dane_dni_tygodnia.columns) <= 3
    prob += lpSum(tablica[(i,osoba)] for i in dane_dni_tygodnia.columns) >= 1 
for osoba in dane_dni_tygodnia.index:
    prob += lpSum(tablica[(i,osoba)] for i in dane_dni_tygodnia.columns if dane_dni_tygodnia.loc[osoba,i] == 0) == 0

# The problem is solved using PuLP's choice of Solver
prob.solve()
    
output = []    
for osoba in dane_dni_tygodnia.index:
    var_output = {
        'Imiona': osoba,
        'Poniedzialek': tablica[('Poniedzialek',osoba)].varValue,
        'Wtorek': tablica[('Wtorek',osoba)].varValue,
        'Sroda': tablica[('Sroda',osoba)].varValue,
        'Czwartek': tablica[('Czwartek',osoba)].varValue,
        'Piatek': tablica[('Piatek',osoba)].varValue
    }
    output.append(var_output)
print(output)

# Na Anakondzie można by zaimportować to do osobnego Excela.
#output_csv = pd.DataFrame(output)
#output_csv.to_csv('pracownicy_gotowe.csv',sep=';',index = False, header=True)

[{'Imiona': 'Ania', 'Poniedzialek': 0.0, 'Wtorek': 0.0, 'Sroda': 0.0, 'Czwartek': 0.0, 'Piatek': 1.0}, {'Imiona': 'Stefan', 'Poniedzialek': 1.0, 'Wtorek': 0.0, 'Sroda': 0.0, 'Czwartek': 0.0, 'Piatek': 0.0}, {'Imiona': 'Hektor', 'Poniedzialek': 0.0, 'Wtorek': 1.0, 'Sroda': 0.0, 'Czwartek': 0.0, 'Piatek': 0.0}, {'Imiona': 'Olaf', 'Poniedzialek': 1.0, 'Wtorek': 0.0, 'Sroda': 0.0, 'Czwartek': 0.0, 'Piatek': 1.0}, {'Imiona': 'Lidia', 'Poniedzialek': 0.0, 'Wtorek': 0.0, 'Sroda': 0.0, 'Czwartek': 1.0, 'Piatek': 1.0}, {'Imiona': 'Piotr', 'Poniedzialek': 0.0, 'Wtorek': 0.0, 'Sroda': 1.0, 'Czwartek': 0.0, 'Piatek': 0.0}]
